In [60]:
import json
from influxdb import InfluxDBClient

In [78]:
env_file = '.env'
with open(env_file) as f:
    env_data = json.load(f)
    
ADDR = env_data["influx_ip"]
USER = env_data["username"]
PASS = env_data["password"]

In [47]:
filename = "./sample.logface"
with open(filename) as f:
    messages = json.load(f)

In [86]:
def message_parser(message):
    """
    Exract info required for data insertion
    """
    
    # Used to remap values if needed
    value_mapping = {
        "on": 1,
        "off": 0
    }
    
    measurement=message["name"]
    displayName=message["displayName"]
    value=message["value"]
    unit=message["unit"]
    did=message["deviceId"]
    timestamp=message["ts"]
    if value in value_mapping.keys():
        value = value_mapping[value]
        
    try:
        value=float(value)
    except ValueError as e:
        # YES I'm passing this - this is because the incoming value is actually a string, let it free
        pass
    
    if type(value) == str:
        line = f'{measurement},deviceId={did} value="{value}",unit="{unit}",displayName="{displayName}" {int(timestamp*1e9)}'
    else:
        line = f'{measurement},deviceId={did} value={value},unit="{unit}",displayName="{displayName}" {int(timestamp*1e9)}'
    print(line)
    return line

In [87]:
def insert_message(message):
    """
    Write the message to the database after parsing
    """
    line = message_parser(message)
    client =InfluxDBClient(
        host=ADDR,
        port=8086,
        username=USER,
        password=PASS)

    client.write([line], {'db': 'hubitat'}, 204, 'line')
    client.close()

In [89]:
# Dumping all the data - good for validating the message parsing.
for i in messages:
    insert_message(i)

battery,deviceId=2 value=6.0,unit="%",displayName="Living room temperature" 1641151260103348224
temperature,deviceId=2 value=66.87,unit="°F",displayName="Living room temperature" 1641151260106056192
humidity,deviceId=2 value=31.3,unit="%",displayName="Living room temperature" 1641151260108071168
pressure,deviceId=2 value=988.0,unit="mbar",displayName="Living room temperature" 1641151260109988096
temperature,deviceId=2 value=66.87,unit="°F",displayName="Living room temperature" 1641151604972479488
humidity,deviceId=2 value=31.5,unit="%",displayName="Living room temperature" 1641151604974621952
pressure,deviceId=2 value=988.2,unit="mbar",displayName="Living room temperature" 1641151604976145920
lastCheckin,deviceId=4 value=1641152137061.0,unit="None",displayName="Rear sliding door" 1641152133546369024
battery,deviceId=4 value=99.0,unit="%",displayName="Rear sliding door" 1641152133548624896
humidity,deviceId=43 value=25.2,unit="%",displayName="Basement temp" 1641153209143397632
temperatu